# Fortran formulae to Python String 

**Synopsis**

Local energy and probability function can be effectively computed using Mathematica. Mathematica can transform formulae into Fortran formula, which is close to syntax of Python, but needs to be slightly modified. Here we modify the output of Mathematica into Python-compatible form. 

In [17]:
import re 

# User definable variables 

**Input**: Set absolute path for the location of the .txt files, containing **Fortran** formulae

In [18]:
probabilityFunctionPath = '/home/degnaiyu/Työpöytä/kanditutkielma/kandityo_koodit/main/Formulae_MathematicaOutput/helium/probabilityFunctionFortranForm_Refine'

localEnergyFunctionPath = '/home/degnaiyu/Työpöytä/kanditutkielma/kandityo_koodit/main/Formulae_MathematicaOutput/helium/localEnergyFortranForm_Refine'

**Input**: set absolute path to where to put the output files 

In [19]:
probabilityOutputFilePath = '/home/degnaiyu/Työpöytä/kanditutkielma/kandityo_koodit/main/Formulae_FortranCompatible/probability_Refine'

localEnergyOutputFilePath = '/home/degnaiyu/Työpöytä/kanditutkielma/kandityo_koodit/main/Formulae_FortranCompatible/localEnergy_Refine'

## Checklist for formula validity 

**Input (optional)** : We expect that the following do not appear in the output of Mathematica: 

In [20]:
checkList = ['Arg']

## Fortran to python dictionary 

**Input (optional)**: Not complete! Only supposed to apply to this project. 

In [21]:
mapping = {
    'E**': 'exp', 
    'Sqrt': 'sqrt', 
    'Abs': 'abs', 
    '\n': '' , 
    "\\" : '', 
    '**':'^', 
}

# Functions 

In [22]:
def readFileToString(absolutePath: str) -> str:  
    '''
    Read file by absolute path. Returns string 
    
    
    Parameters
    ----------
        absolutePath: absolute path of the file 
        
    
    Return
    ------
        Returns the content of the file as string 
    
    '''
    
    file = open(absolutePath, 'r') 
    return file.read()





In [23]:
def checkListFunction(string, checkList: list ) -> list: 
    '''
    Check whether there are unwanted elements in string following the checklist of unwanted elements 
    
    
    Parameters
    ----------
        string: string which we are checking 
        
        
    Return
    ------
        list of unwanted elements in the string
    
    '''
    unwantedList = []
    
    for unwanted in checkList: 
        if unwanted in string: 
            unwantedList.append(unwanted)
            
            
    return unwantedList
            

    
    
    
def printingForUnwantedElements(isLocalEnergy: bool, listOfUnwantedElements: list) -> None: 
    '''
    Printing for unwanted elements in the Mathematica output 
    '''
    
    if isLocalEnergy: 
        prompt = f'Local energy has {len(listOfUnwantedElements)} unwanted elements:'
        print(prompt)
        print(len(prompt)*'-')
        print(listOfUnwantedElements)
    else: 
        prompt = f'Probability function has {len(listOfUnwantedElements)} unwanted elements:'
        print(prompt)
        print(len(prompt)*'-')
        print(listOfUnwantedElements)
        
                

In [24]:
def fortranFormToPythonForm(string: str, mappingDict: dict) -> str: 
    '''
    Transform fortran formula to python formula using the above defined mapping 
    
    
    Parameters
    ---------
        string: input string containing fortran formulae generated by Mathematica 
        
        mappingDict: mapping dictionary, rules for transforming  fortran formulae to python formulae 
        
    Return
    ------
        string which is the input transformed into python formula
    
    '''
    
    transcript = string
    
    # transforming 
    for fortranForm, pythonForm in mappingDict.items(): 
        transcript = transcript.replace(fortranForm, pythonForm)
        
    return transcript
    

In [25]:
def savePythonFormToFile(string: str, absolutePath: str, mode = 'w') -> None:
    '''
    Save string to a file. Can overwrite or append 
    
    Parameters
    ----------
         string: string that is to be written to a file 
         
         absolutePath: filename or file absolute path 
         
         mode: writing mode. Whether to add  or overwrite content to a file. 
         
     Return
     ------
         The file should be created 
    '''
    df = open(absolutePath, mode ) 
    df.write(string )
    df.close()
    
    
    
    
    
    
    
    
    

In [26]:
def findVariables(exprStr: str) -> tuple:
    '''
    Extract variables in expression string. 
    
    Parameters
    ---------
        exprStr: expression as string. Supposed to be Python-compatible formula (at least Fortran, but not tested)
        
    
    Return
    ------
    Tuple of two lists. 
    
        First one: alphanumerically-ordered coordinate list e.g.
        ['x1', 'x2', 'x3', 'y1', 'y2', 'y3', 'z1', 'z2', 'z3']
        
        Second one: alphanumerically-ordered  parameters list 
        e.g. ['A1', 'A2']
        
    '''
    
    patternForCoordinates = r'[xyz]{1}[123]{1}'                   # pattern for: x1, x2, y1 etc. 
    patternForParameters = r'[A-Z]{1}\d{1}'                # pattern for: A1, A2, ... Z9  
    
    coordinateList =sorted(list(set( re.findall(patternForCoordinates, exprStr)  )))
    parametersList =sorted(list(set(  re.findall(patternForParameters, exprStr) )))
    
    return coordinateList, parametersList 
    
    

In [27]:
def regroupToCoordinateTriple_findVariables(coordinateList: list, isCoordinateTripletInTuple: bool = False ) -> list: 
    '''
    Regrouping the coordinates from the output of the previous function `findVariables` into form [(x1, y1, z1), (x2, y2, z2)]
    
    Parameters
    ----------
        coordinateList: output coordinates from findVariables-function 
        
        isCoordinateTripletInTuple: whether the output coordinate triples be separated by parentheses (see Return)
        
    Return
    ------
        list of traditional coordinatetriplets [(x1, y1, z1), (x2, y2, z2)] or in ordered list without parentheses [x1, y1, z1, x2, y2, z2 ]
        
    Warning
    -------
        User check that the coordinateList is the output of the function `findVariables` 
    '''
    
    tripletList = []   # save all triplets 
    
    
    # each particle's components 
    xCoordinates = []
    yCoordinates = []
    zCoordinates = []
    
    
    # separating each particle's component into their lists 
    for component in coordinateList: 
        if 'x' in component: 
            xCoordinates.append(component) 
        elif 'y' in component: 
            yCoordinates.append(component)
        elif 'z' in component:   
            zCoordinates.append(component)
        else: 
            raise Exception("No other than x,y,z components")
    
    
    
    
    if isCoordinateTripletInTuple: 
        # forming triplets 
        for x,y,z in zip(xCoordinates, yCoordinates, zCoordinates): 
            tripletList.append((x,y,z))
    else: 
        # forming ordered list 
        for x,y,z in zip(xCoordinates, yCoordinates, zCoordinates): 
            tripletList.append(x)
            tripletList.append(y)
            tripletList.append(z)
        
        
    return tripletList
    
    
    
    
    

Remove spaces 

In [28]:
def removeSpaces(expr:str): 
    return expr.replace(' ', '')

# Executing the functions 

Comment out if you have fortran forms from Mathematica: 

In [29]:
# read file to strings 


probabilityFunctionString = readFileToString(probabilityFunctionPath)
localEnergyFunctionString = readFileToString(localEnergyFunctionPath)


# check whether the are unwanted elements 
printingForUnwantedElements(
                            isLocalEnergy= False, 
                            listOfUnwantedElements= checkListFunction(string = probabilityFunctionString, checkList=checkList)
)
printingForUnwantedElements(
                            isLocalEnergy= True, 
                            listOfUnwantedElements= checkListFunction(string = localEnergyFunctionString, checkList=checkList)
)


Probability function has 0 unwanted elements:
---------------------------------------------
[]
Local energy has 0 unwanted elements:
-------------------------------------
[]


In [30]:
#transform fortran code to python code 
probability_pythonTransformed = fortranFormToPythonForm(probabilityFunctionString, mappingDict=mapping)
localEnergy_pythonTransformed = fortranFormToPythonForm(localEnergyFunctionString, mappingDict=mapping)

In [31]:
# remove spaces 
probability_pythonTransformed = removeSpaces(probability_pythonTransformed)
localEnergy_pythonTransformed = removeSpaces(localEnergy_pythonTransformed)

In [32]:
# save python-compatible formulae to separate files 
savePythonFormToFile(probability_pythonTransformed, absolutePath=probabilityOutputFilePath)
savePythonFormToFile(localEnergy_pythonTransformed, absolutePath=localEnergyOutputFilePath)

In [33]:
# find coordinates and parameters
coordinates, parameterStringList  = findVariables(localEnergyFunctionString)

# regrouping coordinates 
coordinates = regroupToCoordinateTriple_findVariables(coordinates)


# for functions usage, define list of ordered variables and parameters
mapping =  coordinates + parameterStringList  # global variable: used in probability and local energy functions definition


In [34]:
mapping, len(mapping) 

(['x1', 'y1', 'z1', 'x2', 'y2', 'z2', 'A2', 'A3'], 8)

In [35]:
coordinates, len(coordinates)

(['x1', 'y1', 'z1', 'x2', 'y2', 'z2'], 6)

In [36]:
parameterStringList, len(parameterStringList)

(['A2', 'A3'], 2)

In [37]:
len(probability_pythonTransformed), len(localEnergy_pythonTransformed)

(173, 7996)

# Small testing of eval built-in function 

In [38]:

# if __name__ == "__main__":
    

#     def probabilityfunction(R: list , parametersList: list ):  
#         x1, y1, z1 = R[0]
#         x2, y2, z2 = R[1]
#         x3, y3, z3 = R[2]

#         A1, A2 = parametersList



#         return eval(probability_pythonTransformed)
    
#     import math 
#     value = probabilityfunction(R = [(1,1,1), (2,2,2), (3,3,3)],
#                         parametersList= (1,1) )
#     print(value )